# Feature reduction comparison

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ensure output folder exists
os.makedirs('ArticleFigures', exist_ok=True)

# Traits order 
traits = ['PALK','OLA','ECO','LINO','LINK','MUFA','PUFA']

# ---- Data: Feature-reduction (TPOT) table ----
data = {
    'TPOT 11 Feature': {
        'MAE':  [0.020, 0.152, 0.134, 0.457, 0.253, 0.198, 0.540],
        'RMSE': [0.012, 0.114, 0.116, 0.327, 0.185, 0.203, 0.467],
        'R2':   [0.992, 0.986, 0.971, 0.971, 0.977, 0.860, 0.849]
    },
    'TPOT 8 Feature': {
        'MAE':  [0.020, 0.142, 0.144, 0.452, 0.271, 0.173, 0.511],
        'RMSE': [0.012, 0.103, 0.091, 0.320, 0.173, 0.173, 0.356],
        'R2':   [0.992, 0.988, 0.955, 0.966, 0.980, 0.899, 0.912]
    },
    'TPOT 5 Feature': {
        'MAE':  [0.022, 0.152, 0.147, 0.464, 0.288, 0.180, 0.573],
        'RMSE': [0.018, 0.143, 0.125, 0.458, 0.169, 0.170, 0.446],
        'R2':   [0.983, 0.978, 0.916, 0.930, 0.981, 0.902, 0.862]
    },
    'TPOT 3 Feature': {
        'MAE':  [0.024, 0.160, 0.145, 0.492, 0.304, 0.182, 0.618],
        'RMSE': [0.014, 0.159, 0.181, 0.417, 0.399, 0.105, 0.636],
        'R2':   [0.990, 0.973, 0.823, 0.942, 0.894, 0.963, 0.720]
    }
}

# Convert to long DataFrame
rows = []
for model_name, metrics in data.items():
    for i, tr in enumerate(traits):
        rows.append({
            'Feature reduction': model_name,
            'Trait': tr,
            'MAE': metrics['MAE'][i],
            'RMSE': metrics['RMSE'][i],
            'R2': metrics['R2'][i]
        })
df = pd.DataFrame(rows)

# ---- Styling options ----
palette = ["#05a8aa", "#70d6ff", "#f6caca", "#ff70a6"]

font_name = 'DejaVu Sans'
title_size = 16
axis_label_size = 13
tick_label_size = 10
legend_size = 11
caption_size = 10

# Apply seaborn/matplotlib rcParams
sns.set_style("whitegrid")
sns.set_context("talk", rc={
    "axes.facecolor": "#FFFFFF",
    "font.size": tick_label_size,
    "axes.titlesize": title_size,
    "axes.labelsize": axis_label_size
})
plt.rcParams['font.sans-serif'] = [font_name] + plt.rcParams.get('font.sans-serif', [])
plt.rcParams['legend.frameon'] = True
plt.rcParams['axes.edgecolor'] = '#333333'
plt.rcParams['axes.linewidth'] = 0.8

# Plotting order and labels
models_order = ['TPOT 11 Feature', 'TPOT 8 Feature', 'TPOT 5 Feature', 'TPOT 3 Feature']
metric_names = ['MAE', 'RMSE', 'R2']
ylabels = ['Mean Absolute Error', 'Root Mean Squared Error', 'R²']

# Create figure (3 panels)
fig, axes = plt.subplots(1, 3, figsize=(20, 6), sharey=False)
fig.subplots_adjust(wspace=0.28, top=0.92)

for ax, metric, ylabel in zip(axes, metric_names, ylabels):
    plot_df = df.pivot(index='Trait', columns='Feature reduction', values=metric).loc[traits, models_order]
    x = np.arange(len(traits))
    total_width = 0.82
    n_models = len(models_order)
    bar_width = total_width / n_models
    offsets = np.linspace(-total_width/2 + bar_width/2, total_width/2 - bar_width/2, n_models)

    for i, model in enumerate(models_order):
        ax.bar(x + offsets[i],
               plot_df[model].values,
               width=bar_width,
               label=model,
               color=palette[i % len(palette)],
               edgecolor='#444444',
               linewidth=0.4,
               alpha=1)

    ax.set_xticks(x)
    ax.set_xticklabels(traits, fontsize=tick_label_size)
    ax.set_xlabel('Trait', fontsize=axis_label_size)
    ax.set_ylabel(ylabel, fontsize=axis_label_size)
    ax.set_title(metric, fontsize=title_size, fontweight='semibold')
    ax.grid(axis='y', linestyle='--', linewidth=0.7, alpha=0.6)

    if metric == 'R2':
        ax.set_ylim(0, 1.02)
        ax.yaxis.set_major_locator(plt.MultipleLocator(0.2))

# Legend to the right
legend = axes[-1].legend(title="Feature set", fontsize=legend_size,
                         title_fontsize=legend_size, loc='upper left', bbox_to_anchor=(1.02, 1.0))
legend.get_frame().set_edgecolor('#CCCCCC')
legend.get_frame().set_linewidth(0.6)

# Save high-resolution PNG (600 dpi)
out_path = os.path.join('ArticleFigures', 'FeatureReduction_model_comparison_metrics.png')
fig.savefig(out_path, dpi=600, bbox_inches='tight', facecolor='white')
plt.close(fig)

print(f"Saved figure to: {out_path}")


Saved figure to: ArticleFigures\FeatureReduction_model_comparison_metrics.png
